In [1]:
import os
import re
import tarfile
import pandas as pd
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# ----------------------------
# 1. Setup NLTK
# ----------------------------
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# ----------------------------
# 2. Define file paths (Windows)
# ----------------------------
file_path = r"C:\Users\bbuser\Downloads\aclImdb_v1.tar.gz"
extract_dir = r"C:\Users\bbuser\Downloads\aclImdb"

# ----------------------------
# 3. Extract IMDb dataset (safe mode)
# ----------------------------
if not os.path.exists(extract_dir):
    with tarfile.open(file_path, "r:gz") as tar:
        tar.extractall(path=extract_dir, filter="data")  # safe extraction
    print("✅ Extraction done!")
else:
    print("✅ Already extracted!")

# ----------------------------
# 4. Load dataset into DataFrame
# ----------------------------
def load_imdb_data(data_dir):
    data = {"review": [], "sentiment": []}
    for sentiment in ["pos", "neg"]:
        folder = os.path.join(data_dir, sentiment)
        for fname in os.listdir(folder):
            with open(os.path.join(folder, fname), encoding="utf-8") as f:
                data["review"].append(f.read())
                data["sentiment"].append(sentiment)
    return pd.DataFrame(data)

train_df = load_imdb_data(os.path.join(extract_dir, "aclImdb/train"))
test_df = load_imdb_data(os.path.join(extract_dir, "aclImdb/test"))

df = pd.concat([train_df, test_df], ignore_index=True)
print("Dataset loaded. Shape:", df.shape)

# Optional: for prototyping, sample 10k
# df = df.sample(10000, random_state=42).reset_index(drop=True)

# ----------------------------
# 5. Cleaning Function
# ----------------------------
def clean_review(text):
    # Lowercase
    text = text.lower()
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove URLs/emails
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"\S+@\S+", '', text)
    # Remove non-letters
    text = re.sub(r"[^a-z\s]", '', text)
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords + short tokens
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return " ".join(tokens)

# ----------------------------
# 6. Apply Cleaning
# ----------------------------
print("Cleaning reviews... (this may take a few minutes)")
df["cleaned_review"] = df["review"].apply(clean_review)

# ----------------------------
# 7. Show Examples
# ----------------------------
examples = df[["review", "cleaned_review"]].head(5)
for i, row in examples.iterrows():
    print("\nRaw:", row["review"][:200], "...")
    print("Cleaned:", row["cleaned_review"])

# ----------------------------
# 8. Train/Test Split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    df["cleaned_review"], df["sentiment"], test_size=0.2, random_state=42
)

# ----------------------------
# 9. Feature Extraction (TF-IDF)
# ----------------------------
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# ----------------------------
# 10. Train Model (Logistic Regression)
# ----------------------------
model = LogisticRegression(max_iter=200)
model.fit(X_train_vec, y_train)

# ----------------------------
# 11. Evaluation
# ----------------------------
y_pred = model.predict(X_test_vec)

print("\n✅ Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ----------------------------
# 12. Save Cleaned Dataset
# ----------------------------
output_path = r"C:\Users\bbuser\Downloads\IMDB_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Cleaned dataset saved to {output_path}")




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bbuser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bbuser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bbuser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bbuser\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


✅ Already extracted!
Dataset loaded. Shape: (50000, 2)
Cleaning reviews... (this may take a few minutes)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\bbuser/nltk_data'
    - 'C:\\Users\\bbuser\\anaconda3\\nltk_data'
    - 'C:\\Users\\bbuser\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\bbuser\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\bbuser\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
def load_reviews(folder_path, category, limit=None):
    folder = Path(folder_path)

    files = sorted(folder.glob("*.txt"), key=lambda p: int(p.stem.split("_")[0]))

    rows = []
    for i, file in enumerate(files):
        if limit and i >= limit:   
            break

        file_id, rating = file.stem.split("_")
        review = file.read_text(encoding="utf-8", errors="replace").strip()

        rows.append({
            "id": int(file_id),
            "rating": int(rating),
            "review": review,
            "category": category
        })

    return pd.DataFrame(rows, columns=["id", "rating", "review", "category"]) \
             .sort_values("id") \
             .reset_index(drop=True)


In [ ]:
# Show the first 5 rows in a nice table
train_data.head()
